## Conexión a Google Drive

In [1]:
## Conectar el notebook a googledrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Instalación **RoboFlow** y Configuración de la API


In [2]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninsta

## Importación de Librerías y configuración de la API Key Roiboflow

In [3]:
from roboflow import Roboflow
from PIL import Image, ImageDraw
import json
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [4]:
#API Key RoboFlow
rf = Roboflow(api_key="b5lZRScZzn3NOPhMEVq2")

## Modelo de Identificación de Aisladores en una cadena de Aislamiento

In [5]:
#Carga Modelo Identificación de Objetos (Aisladores en una Cadena)
project_identificacion = rf.workspace().project("estado-aisladores-en-lt")
model_identificacion = project_identificacion.version(1).model

loading Roboflow workspace...
loading Roboflow project...


In [6]:
##imagen a procesar
##imagen_procesar = "DJI_0910.JPG"
##imagen_procesar = "DJI_0149.JPG"
imagen_procesar = "DJI_0117.JPG"

In [7]:
first_folder = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final Maestría/imagenes_identificar/"
image_path = first_folder+imagen_procesar

In [8]:
# visualize your prediction (guardado)
model_identificacion.predict(image_path,
                             confidence=40, overlap=30).save(first_folder+"prediction.jpg")


Extracción de ímagenes a partir de los Bounding Boxes de modelos de identificación

In [9]:
#Predicción y guardado del diccionatio
bounding_boxes_dict = model_identificacion.predict(image_path, confidence=40, overlap=30).json()
#convertir la predicción en formato JSON.
json_data = json.dumps(bounding_boxes_dict)
# Cargar los bounding boxes desde el JSON
data = json.loads(json_data)
bounding_boxes = data["predictions"]
# Cargar la imagen desde la ruta
original_image = Image.open(image_path)

In [10]:
cutted_folder = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final Maestría/imagenes_recortadas/"

# Eliminar el contenido de la carpeta "imagenes_recortadas" antes de guardar nuevas regiones
for file_name in os.listdir(cutted_folder):
    file_path = os.path.join(cutted_folder, file_name)
    if os.path.isfile(file_path):
        os.remove(file_path)

# Iterar a través de los bounding boxes y extraer las regiones (Cada plato de aislamiento)
for i, bbox in enumerate(bounding_boxes, start=1):
    x, y, width, height = bbox["x"], bbox["y"], bbox["width"], bbox["height"]

    # Extraer la región delimitada por el bounding box
    region = original_image.crop((x - width/2, y- height/2, x + width/2, y + height/2))

    # Guardar la región extraída como una nueva imagen
    region_filename = f"region_{i}.jpg"
    region.save(cutted_folder + region_filename)

# Cerrar la imagen original
original_image.close()
region.close()

## Modelo de Segmentación de los Aisladores usando Segment Anything By Meta.

####Environment Set-up

In [11]:
using_colab = True

In [12]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/groceries.jpg

    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.1.0+cu118
Torchvision version: 0.16.0+cu118
CUDA is available: True
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-w6vpcqaq
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-w6vpcqaq
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36586 sha256=42e1f1ec8518db5b58cd8f75079c0046b34423e2692105ef5a9d6a24ba1e91bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-67dxhy87/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything
--2023-10-22 01:58:07--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.c

In [13]:
#load the SAM model and predictor
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

####Segmentación de imágenes (platos de aislamiento)

In [14]:
 # Función para segmentar las imagenes recortadas. A cada imágen se le extrae solo el plato de aislamiento (quitar ruido ambiente) y se gurda en la carpeta imagenes_segmentadas

def segment_class(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  #Process the image to produce an image embedding by calling SamPredictor.set_image.
  predictor.set_image(image)
  # Obtener las dimensiones de la imagen
  height, width, _ = image.shape
  #Calcular el punto medio en las coordenadas (X, Y)
  mid_x = width/2
  mid_y = height/4
  #Obtenemos los puntos para usar Segment Anything
  input_point = np.array([[mid_x, mid_y]])
  input_label = np.array([1])
  #Predicción para obtener la máscara
  masks, scores, logits = predictor.predict(
      point_coords=input_point,
      point_labels=input_label,
      multimask_output=False,
  )
  #
  mask = np.uint8(masks)
  # Ajustar el tamaño de la máscara para que coincida con las dimensiones de la imagen
  mask_resized = cv2.resize(mask[0], (image.shape[1], image.shape[0]))
  # Aplicar la máscara a la imagen original
  segmented_image = cv2.bitwise_and(image, image, mask=mask_resized)
  # Guardar la región extraída como una nueva imagen

  return segmented_image

In [15]:
# folder de imágenes segementadas
segment_folder = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final Maestría/imagenes_segmentadas/"

# folder de imágenes recortadas
cutted_folder = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final Maestría/imagenes_recortadas/"

In [16]:
# Obtener la lista de nombres de archivos en la carpeta de entrada
file_names = os.listdir(cutted_folder)

# Eliminar el contenido de la carpeta "imagenes_segmentadas" antes de guardar las nuevas imagenes segmentadas
for file_name in os.listdir(segment_folder):
    file_path = os.path.join(segment_folder, file_name)
    if os.path.isfile(file_path):
        os.remove(file_path)

# Iterar sobre cada archivo en la carpeta de entrada
for file_name in file_names:
    # Construir la ruta completa de la imagen de entrada
    input_image_path = os.path.join(cutted_folder, file_name)

    # Leer la imagen de entrada
    image = cv2.imread(input_image_path)

    # Aplicar la función segment_class para obtener la imagen segmentada
    segmented_image = segment_class(image)

    # Construir el nombre del archivo de salida
    output_file_name = f"{os.path.splitext(file_name)[0]}_segmentada.jpg"

    # Construir la ruta completa de la imagen de salida
    segment_image_path = os.path.join(segment_folder, output_file_name)

    # Guardar la imagen segmentada en la carpeta de salida
    cv2.imwrite(segment_image_path, segmented_image)

    print(f"Imagen segmentada guardada: {output_file_name}")


Imagen segmentada guardada: region_1_segmentada.jpg
Imagen segmentada guardada: region_5_segmentada.jpg
Imagen segmentada guardada: region_2_segmentada.jpg
Imagen segmentada guardada: region_3_segmentada.jpg
Imagen segmentada guardada: region_7_segmentada.jpg
Imagen segmentada guardada: region_6_segmentada.jpg
Imagen segmentada guardada: region_4_segmentada.jpg
Imagen segmentada guardada: region_8_segmentada.jpg


## Modelo de Clasificación de Aisladores Segmentados según su estado

In [17]:
project = rf.workspace().project("clasificacion-del-ailamiento-en-lt")
model = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...


In [18]:
# folder de imágenes clasificadas
class_folder = "/content/drive/MyDrive/Colab Notebooks/Proyecto Final Maestría/imagenes_clasificadas/"

In [19]:
# Obtener la lista de nombres de archivos en la carpeta de segmentación
file_names = os.listdir(segment_folder)

In [20]:
# Eliminar el contenido de la carpeta "imagenes_clasificadas" antes de guardar las nuevas imagenes segmentadas
for file_name in os.listdir(class_folder):
    file_path = os.path.join(class_folder, file_name)
    if os.path.isfile(file_path):
        os.remove(file_path)

# Iterar sobre cada archivo en la carpeta de segmentación
for file_name in file_names:
    # Construir la ruta completa de la imagen segmentada
    input_image_path = os.path.join(segment_folder, file_name)

    # Aplicar la predicción de Roboflow a la imagen segmentada
    response = model.predict(input_image_path)

    # Construir el nombre del archivo de salida
    output_file_name = f"{os.path.splitext(file_name)[0]}_predict.jpg"

    # Construir la ruta completa de la imagen de salida
    output_image_path = os.path.join(class_folder, output_file_name)

    # Guardar la imagen resultante en la carpeta de salida
    response.save(output_image_path)

    print(f"Imagen clasificada guardada: {output_file_name}")



Imagen clasificada guardada: region_1_segmentada_predict.jpg
Imagen clasificada guardada: region_5_segmentada_predict.jpg
Imagen clasificada guardada: region_3_segmentada_predict.jpg
Imagen clasificada guardada: region_2_segmentada_predict.jpg
Imagen clasificada guardada: region_7_segmentada_predict.jpg
Imagen clasificada guardada: region_6_segmentada_predict.jpg
Imagen clasificada guardada: region_4_segmentada_predict.jpg
Imagen clasificada guardada: region_8_segmentada_predict.jpg
